| Column Name             | Data Type | Description                                              | null |
|-------------------------|-----------|----------------------------------------------------------|---|
| Code                    | int64     | レースの識別番号                                         |0|
| Rank                    | int64     | レースにおける順位                                       |0|
| Frame Rank              | int64     | フレーム内での順位                                       |0|
| Horse Number            | int64     | 馬の番号                                                 |0|
| Horse Name              | object    | 馬の名前                                                 |0|
| Kinryou                 | float64   | 馬の斤量（体重）                                       |0|
| Jockey                  | object    | ジョッキーの名前                                         |0|
| Time                    | float64   | レースの所要時間（秒数）                                |0|
| Chakusa                 | object    | 着順（フォーマットによる）                               |904084|
| Nobori                  | float64   | 上昇度（成績の向上を示す指標）                         |2021595|
| Tansyou                 | float64   | 単勝オッズ                                               |1659530|
| Ninki                   | float64   | 人気（馬の人気度）                                      |46|
| Horse Weight            | float64   | 馬の体重（斤量）                                       |3547|
| Trainer                 | object    | トレーナーの名前                                        |0|
| Banushi                 | object    | 馬主の名前                                              |96709|
| Shoukin                 | object    | 賞金                                                    |3167645|
| Date                    | object    | レース日（通常は文字列形式）                            |0|
| Track                   | object    | トラックの種類                                          |0|
| Weather                 | object    | 天候                                                    |0|
| Race Number             | int64     | レースの番号                                            |0|
| Distance                | int64     | レースの距離（メートル単位）                            |0|
| Condition               | object    | トラックコンディション                                   |387819|
| Disqualification        | int64     | 失格の有無（失格の場合は1、そうでなければ0）           |0|
| Sex                     | int64     | 馬の性別（数値で表現）                                  |0|
| Age                     | int64     | 馬の年齢（数値で表現）                                  |0|
| Track Type              | object    | トラックのタイプ                                        |0|
| intermediate_rank_1     | float64   | 中間順位1                                               |1072605|
| intermediate_rank_2     | float64   | 中間順位2                                               |1116477|
| intermediate_rank_3     | float64   | 中間順位3                                               |2367017|
| intermediate_rank_4     | float64   | 中間順位4                                               |3182497|
| Horse Weight Change      | int64     | 馬の体重の変化（斤量の増減）                           |0|
| Trainers Stable         | object    | トレーナーの厩舎名                                      |0|

5886227行

In [1]:
import tensorflow_probability as tfp
import tensorflow as tf

2024-11-05 04:32:27.542298: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-05 04:32:27.565528: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-05 04:32:27.660254: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-05 04:32:27.852698: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730748747.941643   69690 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730748747.97

In [2]:
import pandas as pd
merged_table = pd.read_csv('./data/merged_table_finished2.csv')

In [3]:
# Chakusa, Nobori, Tansyou, Banushi, Shoukin, intermediate_rank_1, intermediate_rank_2, intermediate_rank_3, intermediate_rank_4のカラカラムは削除
merged_table = merged_table.drop(['Chakusa', 'Nobori', 'Tansyou', 'Banushi', 'Shoukin', 'intermediate_rank_1', 'intermediate_rank_2', 'intermediate_rank_3', 'intermediate_rank_4'], axis=1)

# Ninki, Horse Weightカラムの欠損行を削除
merged_table = merged_table.dropna(subset=['Ninki', 'Horse Weight'])

# Conditionのnullは0に変換
merged_table['Condition'] = merged_table['Condition'].fillna(0)

In [4]:
# 全列表示
pd.set_option('display.max_columns', None)
merged_table.head()

,Code,Rank,Frame Rank,Horse Number,Horse Name,Kinryou,Jockey,Time,Ninki,Horse Weight,Trainer,Date,Track,Weather,Race Number,Distance,Condition,Disqualification,Sex,Age,Track Type,Horse Weight Change,Trainers Stable
0,200542011402,1,2,2,イッチョルノヨ,54.0,加藤和博,49.0,4.0,491.0,阿部秀一,2005/01/14,浦和,晴,2,800,良,0,1,3,ダート,0,地
1,200542011402,2,3,3,チャンピオンリボン,53.0,御神本訓,49.3,1.0,500.0,須藤一弘,2005/01/14,浦和,晴,2,800,良,0,2,3,ダート,0,地
2,200542011402,3,6,6,ホリノケラケラ,54.0,橋本直哉,50.2,6.0,488.0,内野健二,2005/01/14,浦和,晴,2,800,良,0,1,3,ダート,0,地
3,200542011402,4,1,1,アイビーレジェンド,53.0,平山真希,50.6,2.0,480.0,内田友一,2005/01/14,浦和,晴,2,800,良,0,2,3,ダート,0,地
4,200542011402,5,7,7,センショウ,54.0,岡田大,50.7,5.0,516.0,藤本昌行,2005/01/14,浦和,晴,2,800,良,0,1,3,ダート,0,地


In [5]:
# str型のカテゴリカルデータを数値に変換
# 1. Track
# 札幌|函館|福島|新潟|東京|中山|中京|京都|阪神|小倉|門別|盛岡|水沢|浦和|船橋|大井|川崎|金沢|笠松|名古屋|園田|姫路|高知|佐賀|帯広
# 0...24

merged_table['Track'] = merged_table['Track'].replace('札幌', 0)
merged_table['Track'] = merged_table['Track'].replace('函館', 1)
merged_table['Track'] = merged_table['Track'].replace('福島', 2)
merged_table['Track'] = merged_table['Track'].replace('新潟', 3)
merged_table['Track'] = merged_table['Track'].replace('東京', 4)
merged_table['Track'] = merged_table['Track'].replace('中山', 5)
merged_table['Track'] = merged_table['Track'].replace('中京', 6)
merged_table['Track'] = merged_table['Track'].replace('京都', 7)
merged_table['Track'] = merged_table['Track'].replace('阪神', 8)
merged_table['Track'] = merged_table['Track'].replace('小倉', 9)
merged_table['Track'] = merged_table['Track'].replace('門別', 10)
merged_table['Track'] = merged_table['Track'].replace('盛岡', 11)
merged_table['Track'] = merged_table['Track'].replace('水沢', 12)
merged_table['Track'] = merged_table['Track'].replace('浦和', 13)
merged_table['Track'] = merged_table['Track'].replace('船橋', 14)
merged_table['Track'] = merged_table['Track'].replace('大井', 15)
merged_table['Track'] = merged_table['Track'].replace('川崎', 16)
merged_table['Track'] = merged_table['Track'].replace('金沢', 17)
merged_table['Track'] = merged_table['Track'].replace('笠松', 18)
merged_table['Track'] = merged_table['Track'].replace('名古屋', 19)
merged_table['Track'] = merged_table['Track'].replace('園田', 20)
merged_table['Track'] = merged_table['Track'].replace('姫路', 21)
merged_table['Track'] = merged_table['Track'].replace('高知', 22)
merged_table['Track'] = merged_table['Track'].replace('佐賀', 23)
merged_table['Track'] = merged_table['Track'].replace('帯広', 24)

array(['浦和', '笠松', '帯広', '園田', '中山', '京都', '水沢', '高知', '佐賀', '大井', '名古屋',
       '船橋', '金沢', '門別', '盛岡', '阪神', '新潟', '東京', '川崎', '福島', '函館', '中京',
       '小倉', '姫路', '札幌'], dtype=object)

In [ ]:
merged_table['Track'].unique()